In [3]:
from PIL import Image
import os

directory = 'red-blue'

# Open a file to write the COE data
with open('pokemon_sprites.coe', 'w') as coe_file:
    coe_file.write("memory_initialization_radix=16;\n")
    coe_file.write("memory_initialization_vector=\n")

    # Iterate over each file in the directory
    for filename in os.listdir(directory):
        if filename.lower().endswith('.png'):
            img_path = os.path.join(directory, filename)
            img = Image.open(img_path)

            img = img.convert('RGB')

            for y in range(img.height):
                for x in range(img.width):
                    r, g, b = img.getpixel((x, y))
                    
                    # Perform color quantization if necessary
                    r >>= 4
                    g >>= 4
                    b >>= 4

                    hex_value = f"{r:01X}{g:01X}{b:01X}"

                    coe_file.write(hex_value + ",\n")

            # Add a delimiter between sprites
            coe_file.write("000,\n")  # Assuming 000 is not a valid sprite pixel and can act as a delimiter

    coe_file.write(";")
